# La Commune de Paris 1871 en liberté - Faire une requête sur wikidata en Python

Ce chapitre documente un point technique important pour la suite. Il explique comment le code fonctionne, et n'interroge pas la pertinance des choix de la requête. Cela est vu à d'autres moment.

L'exemple est pris sur la recherche des communard·e·s présent·e·s dans Wikidata. L'explication de qu'est ce qu'un·e "communard·e·s présent·e·s dans Wikidata" sera vu plus loin.

### Méthode

La classe Extraction_wikidata, est une adaption du modèle de réquête en python qui se trouve sur https://query.wikidata.org/

- dans la variable query, il faut copier/coller la requête faite dans query.wikidata.org
- exécuter les cellules "## 0" à "## 5" comme si dessous afin d'obtenir le résultat dans un dataframe exploitable pour faire des manipulations de données.

Cela est reproductible, et l'est à de nombreuses occurence dans le document, sur plusieurs requêtes. 
Il suffit de changer la valeur de query (en ## 1) puis personnaliser les ##4 et 5 pour obtenir un tableau (dataframe) exploitable.

In [1]:
# les installations nécessaires
#https://rdflib.github.io/sparqlwrapper/
#!pip install sparqlwrapper
#!pip install seaborn

In [2]:
## 0 - import
import sys

from SPARQLWrapper import SPARQLWrapper, JSON

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import HTML

In [3]:
## 1 - les variables
endpoint_url:str = "https://query.wikidata.org/sparql"

In [4]:
query:str = """SELECT ?communard_ou_communarde ?communard_ou_communardeLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }
  ?communard_ou_communarde wdt:P106 wd:Q1780490.
}"""

In [5]:
## 2 - l'objet
class Extraction_wikidata():
    def __init__(self, endpoint_url:str, query:str):
        self.endpoint_url:str = endpoint_url
        self.query:str = query
        
    def get_results(self, endpoint_url:str, query:str):
        user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        # TODO adjust user agent; see https://w.wiki/CX6
        sparql = SPARQLWrapper(endpoint_url, agent = user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()
    
    def json_to_df(self, entree):
        #results = sparql.query().convert()
        df_results = pd.json_normalize(entree["results"]["bindings"])
        return df_results
    
    def extraire_et_df(self):
        extract_json = self.get_results(self.endpoint_url, self.query)
        extract_df = self.json_to_df(extract_json)
        return extract_df


In [6]:
## 3 - instantiation de l'objet et récupération du df
extraction_communard = Extraction_wikidata(endpoint_url, query)
df_communard = extraction_communard.extraire_et_df()

In [7]:
## 4 - afficher le df voulu
print(df_communard)

    communard_ou_communarde.type              communard_ou_communarde.value  \
0                            uri      http://www.wikidata.org/entity/Q20951   
1                            uri      http://www.wikidata.org/entity/Q34618   
2                            uri     http://www.wikidata.org/entity/Q184535   
3                            uri     http://www.wikidata.org/entity/Q216092   
4                            uri     http://www.wikidata.org/entity/Q257950   
..                           ...                                        ...   
304                          uri  http://www.wikidata.org/entity/Q123735409   
305                          uri  http://www.wikidata.org/entity/Q123735410   
306                          uri  http://www.wikidata.org/entity/Q123735411   
307                          uri  http://www.wikidata.org/entity/Q123735412   
308                          uri  http://www.wikidata.org/entity/Q123735413   

    communard_ou_communardeLabel.xml:lang communard

In [11]:
## 5 - quelques statistiques sur le df
#df_communard.describe()

In [13]:
#df_communard_en_html = df_communard.to_html(columns = ['communard_ou_communarde.value', 'communard_ou_communardeLabel.value'])
#HTML(df_communard_en_html)

### Avoir TOUTES les informations sur les communard·e·s

Cette requête, qui fonctionne comme expliqué ci dessus est importante. Car en plus de charger toutes les personnes concernées (leur label) elle récupére toutes les données liées à leur élément.

Ce dataframe sera ré-utilisé à de nombreuses occasions.

Pour toutes les personnes, chercher les ?p ?q avec gestion des labels.
Attention : c'est lourd !

#### Aller chercher les valeurs et les mettre dans un dataframe (df) nommé df_communard_tous_p_q

In [14]:
query = """SELECT ?communard_ou_communarde ?communard_ou_communardeLabel ?p ?pLabel ?q ?qLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en".
                         ?communard_ou_communarde rdfs:label ?communard_ou_communardeLabel.
                         ?p rdfs:label ?pLabel.
                         ?q rdfs:label ?qLabel.}
  
  ?communard_ou_communarde wdt:P106 wd:Q1780490;
                           ?p ?q. 
}
"""

In [15]:
extraction_communard_tous_p_q = Extraction_wikidata(endpoint_url, query)
df_communard_tous_p_q = extraction_communard_tous_p_q.extraire_et_df()

In [16]:
df_communard_tous_p_q

,communard_ou_communarde.type,communard_ou_communarde.value,p.type,p.value,q.xml:lang,q.type,q.value,communard_ou_communardeLabel.xml:lang,communard_ou_communardeLabel.type,communard_ou_communardeLabel.value,pLabel.type,pLabel.value,qLabel.type,qLabel.value,q.datatype,qLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q20951,uri,http://www.w3.org/2004/02/skos/core#altLabel,oc,literal,Elisèu Reclus,fr,literal,Élisée Reclus,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Elisèu Reclus,NaN,NaN
1,uri,http://www.wikidata.org/entity/Q20951,uri,http://www.w3.org/2004/02/skos/core#altLabel,id,literal,Jacques Élisée Reclus,fr,literal,Élisée Reclus,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Jacques Élisée Reclus,NaN,NaN
2,uri,http://www.wikidata.org/entity/Q20951,uri,http://www.w3.org/2004/02/skos/core#altLabel,sv,literal,Jean Jacques Élisée Reclus,fr,literal,Élisée Reclus,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Jean Jacques Élisée Reclus,NaN,NaN
3,uri,http://www.wikidata.org/entity/Q20951,uri,http://www.w3.org/2004/02/skos/core#altLabel,en,literal,Jacques Élisée Reclus,fr,literal,Élisée Reclus,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Jacques Élisée Reclus,NaN,NaN
4,uri,http://www.wikidata.org/entity/Q20951,uri,http://www.w3.org/2004/02/skos/core#altLabel,ast,literal,Jacques Élisée Reclus,fr,literal,Élisée Reclus,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Jacques Élisée Reclus,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34314,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/P18,NaN,uri,http://www.wikidata.org/entity/statement/Q3035...,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/P18,literal,statement/Q3035446-B6A89595-3458-4BB4-A585-269...,NaN,NaN
34315,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/P19,NaN,uri,http://www.wikidata.org/entity/statement/q3035...,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/P19,literal,statement/q3035446-26BEF6FE-AC9A-4BBC-A4C7-F0C...,NaN,NaN
34316,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/P20,NaN,uri,http://www.wikidata.org/entity/statement/Q3035...,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/P20,literal,statement/Q3035446-A1D1C4ED-3E9B-4B9E-919C-977...,NaN,NaN
34317,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/P21,NaN,uri,http://www.wikidata.org/entity/statement/Q3035...,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/P21,literal,statement/Q3035446-E073AA93-5474-4D10-9AF9-0D0...,NaN,NaN


#### Quelques apperçu de travail sur ce df

In [17]:
#df_communard_tous_p_q.describe()

In [18]:
df_communard_tous_p_q.columns

Index(['communard_ou_communarde.type', 'communard_ou_communarde.value',
       'p.type', 'p.value', 'q.xml:lang', 'q.type', 'q.value',
       'communard_ou_communardeLabel.xml:lang',
       'communard_ou_communardeLabel.type',
       'communard_ou_communardeLabel.value', 'pLabel.type', 'pLabel.value',
       'qLabel.type', 'qLabel.value', 'q.datatype', 'qLabel.xml:lang'],
      dtype='object')

In [19]:
df_communard_tous_p_q.tail(25)

,communard_ou_communarde.type,communard_ou_communarde.value,p.type,p.value,q.xml:lang,q.type,q.value,communard_ou_communardeLabel.xml:lang,communard_ou_communardeLabel.type,communard_ou_communardeLabel.value,pLabel.type,pLabel.value,qLabel.type,qLabel.value,q.datatype,qLabel.xml:lang
34294,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.w3.org/2004/02/skos/core#altLabel,en,literal,Dominique Regere,fr,literal,Dominique Régère,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Dominique Regere,NaN,NaN
34295,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.w3.org/2004/02/skos/core#altLabel,fr,literal,Dominique Théophile Régère,fr,literal,Dominique Régère,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Dominique Théophile Régère,NaN,NaN
34296,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.w3.org/2004/02/skos/core#altLabel,nl,literal,Dominique Regere,fr,literal,Dominique Régère,literal,http://www.w3.org/2004/02/skos/core#altLabel,literal,Dominique Regere,NaN,NaN
34297,uri,http://www.wikidata.org/entity/Q3035446,uri,http://wikiba.se/ontology#statements,NaN,literal,32,fr,literal,Dominique Régère,literal,http://wikiba.se/ontology#statements,literal,32,http://www.w3.org/2001/XMLSchema#integer,NaN
34298,uri,http://www.wikidata.org/entity/Q3035446,uri,http://wikiba.se/ontology#sitelinks,NaN,literal,3,fr,literal,Dominique Régère,literal,http://wikiba.se/ontology#sitelinks,literal,3,http://www.w3.org/2001/XMLSchema#integer,NaN
34299,uri,http://www.wikidata.org/entity/Q3035446,uri,http://wikiba.se/ontology#identifiers,NaN,literal,9,fr,literal,Dominique Régère,literal,http://wikiba.se/ontology#identifiers,literal,9,http://www.w3.org/2001/XMLSchema#integer,NaN
34300,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/direct/P18,NaN,uri,http://commons.wikimedia.org/wiki/Special:File...,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/direct/P18,literal,http://commons.wikimedia.org/wiki/Special:File...,NaN,NaN
34301,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/direct/P19,NaN,uri,http://www.wikidata.org/entity/Q1479,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/direct/P19,literal,Bordeaux,NaN,fr
34302,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/direct/P20,NaN,uri,http://www.wikidata.org/entity/Q163948,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/direct/P20,literal,10e arrondissement de Paris,NaN,fr
34303,uri,http://www.wikidata.org/entity/Q3035446,uri,http://www.wikidata.org/prop/direct/P21,NaN,uri,http://www.wikidata.org/entity/Q6581097,fr,literal,Dominique Régère,literal,http://www.wikidata.org/prop/direct/P21,literal,masculin,NaN,fr


#### Nettoyer le df et le mettre dans df_tout_sur_tous_communard

In [20]:
# retirer les colonnes "inutiles"
df_nettoyage_colonne = df_communard_tous_p_q.loc[:,['communard_ou_communarde.value','p.value','q.value','communard_ou_communardeLabel.xml:lang','communard_ou_communardeLabel.value', 'pLabel.value','qLabel.value', 'q.xml:lang']]
# garder le français
df_nettoyage_langue = df_nettoyage_colonne[df_nettoyage_colonne['q.xml:lang'] == ("fr" or "NaN")]

In [21]:
df_nettoyage_langue.describe()

,communard_ou_communarde.value,p.value,q.value,communard_ou_communardeLabel.xml:lang,communard_ou_communardeLabel.value,pLabel.value,qLabel.value,q.xml:lang
count,1072,1072,1072,1072,1072,1072,1072,1072
unique,309,10,774,1,309,10,774,1
top,http://www.wikidata.org/entity/Q309722,http://www.w3.org/2000/01/rdf-schema#label,communarde,fr,Auguste Blanqui,http://www.w3.org/2000/01/rdf-schema#label,communarde,fr
freq,11,309,47,1072,11,309,47,1072


In [22]:
# Avoir un df exploitable pour la suite
df_tout_sur_tous_communard = df_nettoyage_langue